In [1]:
!pip install html-table-parser-python3
!pip install xlsxwriter

In [2]:
import time
#time.sleep(5)

In [4]:
#mounting drive
#from google.colab import drive
#drive.mount('/content/drive')

#import os
#os.chdir('/content/drive/MyDrive/codes')


# from google.colab import files
# uploaded = files.upload()

In [4]:
####importing csv to pandas and finally to list###########
# importing module
import pandas as pd

# reading CSV file
#autourl =pd.read_csv("/content/drive/MyDrive/codes/ev_benchmark_url.csv",header=None)
autourl =pd.read_excel("url.xlsx",header=None)
print(autourl.head(5))
ev_list=autourl[0].tolist()

                                                   0
0  https://ev-database.org/car/2260/Opel-Zafira-e...
1  https://ev-database.org/car/2261/Opel-Zafira-e...
2  https://ev-database.org/car/2273/Audi-S6-Avant...
3  https://ev-database.org/car/2272/Audi-A6-Avant...
4  https://ev-database.org/car/2271/Audi-S6-Sport...


In [5]:
#######create excel sheet##################
# import xlsxwriter module
import xlsxwriter
workbook = xlsxwriter.Workbook('ev_database_1.xlsx')
workbook.close()

In [10]:
# Library for opening url and creating
# requests
import urllib.request

# pretty-print python data structures
from pprint import pprint

# for parsing all the tables present
# on the website
from html_table_parser.parser import HTMLTableParser

# for converting the parsed data in a
# pandas dataframe
import pandas as pd


# Opens a website and read its
# binary contents (HTTP Response Body)
def url_get_contents(url):

	# Opens a website and read its
	# binary contents (HTTP Response Body)

	#making request to the website
	req = urllib.request.Request(url=url)
	f = urllib.request.urlopen(req)

	#reading contents of the website
	return f.read()
#########outide loop variable#################
data_bin=[]
out_bin=[]

###########################
#ev_list=['https://ev-database.org/car/1651/Volkswagen-ID-Buzz-Pro','https://ev-database.org/car/1240/Mercedes-EQV-300-Long']
#ev_list=['https://ev-database.org/car/1240/Mercedes-EQV-300-Long']
for j in range(0,len(ev_list)):
  # defining the html contents of a URL.
  url=ev_list[j]
  xhtml = url_get_contents(url).decode('utf-8')

  # Defining the HTMLTableParser object
  p = HTMLTableParser()

  # feeding the html contents in the
  # HTMLTableParser object
  p.feed(xhtml)

  # Now finally obtaining the data of
  # the table required
  #pprint(len(p.tables))
  ########################################
  ################################################
  out=url.split('/')[-1]
  ###removing special character with underscore######
  out= out.translate ({ord(c): "_" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
  out_bin.append(out)
  ###################################################
  ##############################################
  data=pd.DataFrame({'0': [out]})
  
  data=data.append(p.tables[0])
  data.reset_index(drop=True)
  for i in range(0,len(p.tables)-1):
    # print(i)
    # pprint(p.tables[i])
    # data=pd.DataFrame(p.tables[i])
    data=data.append(pd.DataFrame(p.tables[i+1]))
  data.reset_index(drop=True)

  ############reducing name to 30 characters###########
  # if(len(out)>29):
  #   out=out[0:29]
  #####################################################
  data_bin.append(data)
  
  ###################################################
  #time.sleep(15)


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# ############exporting into excel with units####################
# startrow = 0
# import os
# path = r"/content/drive/MyDrive/codes/ev_database.xlsx"
# writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
# for k in range(0,len(data_bin)):
#   data_bin[k].to_excel(writer, sheet_name =str(k),startrow=startrow,header=False,index=False)
# writer.save()

**Units Remove**

In [25]:
#########exporting data without units########## 
for n in range(0,len(data_bin)):
  data_bin[n].columns =['name', 'value','nothing1','nothing2','nothing3','nothing4']
  data_bin[n].value = data_bin[n].value.str.extract('(\d*\.\d+|\d+)', expand=False).astype(float)
data_bin[1]


,name,value,nothing1,nothing2,nothing3,nothing4
0,NaN,NaN,NaN,NaN,NaN,Mercedes_EQV_300_Long
0,United Kingdom,83.0,NaN,NaN,NaN,NaN
1,The Netherlands,80.0,NaN,NaN,NaN,NaN
2,Germany,71.0,NaN,NaN,NaN,NaN
0,United Kingdom,2020.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2,CCS (150 kW DC),110.0,96 kW,41 min,310 km/h,NaN
0,Charge Curve,NaN,NaN,NaN,NaN,NaN
1,,NaN,NaN,NaN,NaN,NaN
2,Data made available by Fastned,NaN,NaN,NaN,NaN,NaN


In [ ]:
############exporting into excel####################
startrow = 0
import os
path = r"/content/drive/MyDrive/codes/ev_database.xlsx"
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
for k in range(0,len(data_bin)):
  data_bin[k].to_excel(writer, sheet_name =str(k),startrow=startrow,header=False,index=False)
writer.save()

**Consolidating data**

In [ ]:
listy = [[] for i in range(0,len(data_bin))]
print(listy)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [ ]:
ev_vardf =pd.read_csv("/content/drive/MyDrive/codes/1_Variables_EV_BENCHMARK.csv",header=None)
vari=ev_vardf[0].tolist()

In [ ]:
# vari=["Range","Top"]
for m in vari:
  for n in range(0,len(data_bin)):
    try:
      temp=data_bin[n].loc[data_bin[n].name.str.contains(m)==True]
      if(len(temp.value)>1):
        listy[n].append("duplicate")
      else:      
        listy[n].append(temp.value.iloc[0])
    except:
      listy[n].append("NA")

In [ ]:
print(listy)

[[10.2, 145.0, 330.0, 150.0, 310.0, nan, 82.0, 77.0, 2.0, 'duplicate', 11.0, 8.0, 40.0, nan, nan, 175.0, 30.0, 460.0, 4712.0, 'duplicate', 2212.0, 1927.0, 2989.0, 2486.0, 3000.0, 589.0, 'duplicate', 2123.0, 0.0, 100.0, nan, 750.0, 1000.0, 50.0, 5.0, nan, 11.1, 'duplicate', nan, nan, nan, nan], [12.1, 160.0, 305.0, 150.0, 362.0, nan, 100.0, 90.0, 2.0, 'duplicate', 11.0, 9.0, 32.0, nan, nan, 110.0, 41.0, 310.0, 5140.0, 'duplicate', 2244.0, 1901.0, 3200.0, 2635.0, 3500.0, 940.0, 'duplicate', 4630.0, nan, nan, nan, 0.0, 0.0, 0.0, 7.0, nan, 11.8, 'duplicate', nan, nan, nan, nan], [13.1, 130.0, 260.0, 100.0, 260.0, nan, 75.0, 68.0, 2.0, 'duplicate', 7.4, 11.0, 24.0, nan, nan, 100.0, 38.0, 280.0, 5306.0, 'duplicate', 2204.0, 1890.0, 3275.0, 2167.0, 3100.0, 1008.0, 'duplicate', 3497.0, 0.0, 0.0, nan, 750.0, 1000.0, 60.0, 8.0, 3.0, 12.4, 'duplicate', nan, nan, nan, nan], [14.3, 130.0, 265.0, 100.0, 260.0, nan, 75.0, 68.0, 2.0, 'duplicate', 7.4, 11.0, 25.0, nan, nan, 100.0, 38.0, 290.0, 4956.0, 

In [ ]:
print(vari)

['Acceleration 0 - 100 km/h', 'Top Speed', 'Electric Range', 'Total Power', 'Total Torque', 'Drive', 'Battery Capacity', 'Battery Useable', 'Charge Port', 'Port Location', 'Charge Power', 'Charge Time ', 'Charge Speed', 'Fastcharge Port', 'FC Port Location', 'Fastcharge Power', 'Fastcharge Time ', 'Fastcharge Speed', 'Length', 'Width', 'Width with mirrors', 'Height', 'Wheelbase', 'Weight Unladen', 'Gross Vehicle Weight', 'Max. Payload', 'Cargo Volume', 'Cargo Volume Max', 'Cargo Volume Frunk', 'Roof Load', 'Tow Hitch Possible', 'Towing Weight Unbraked', 'Towing Weight Braked', 'Vertical Load Max', 'Seats', 'Isofix', 'Turning Circle', 'Platform', 'Car Body', 'Segment', 'Roof Rails', 'EV Dedicated Platform']


In [ ]:
df = pd.DataFrame(listy,columns=vari)
df=df.T
df.columns=out_bin

In [ ]:
print(df)

                          Volkswagen_ID_Buzz_Pro Mercedes_EQV_300_Long  \
Acceleration 0 - 100 km/h                   10.2                  12.1   
Top Speed                                  145.0                 160.0   
Electric Range                             330.0                 305.0   
Total Power                                150.0                 150.0   
Total Torque                               310.0                 362.0   
Drive                                        NaN                   NaN   
Battery Capacity                            82.0                 100.0   
Battery Useable                             77.0                  90.0   
Charge Port                                  2.0                   2.0   
Port Location                          duplicate             duplicate   
Charge Power                                11.0                  11.0   
Charge Time                                  8.0                   9.0   
Charge Speed                          

In [ ]:
print(vari)

['Acceleration 0 - 100 km/h', 'Top Speed', 'Electric Range', 'Total Power', 'Total Torque', 'Drive', 'Battery Capacity', 'Battery Useable', 'Charge Port', 'Port Location', 'Charge Power', 'Charge Time ', 'Charge Speed', 'Fastcharge Port', 'FC Port Location', 'Fastcharge Power', 'Fastcharge Time ', 'Fastcharge Speed', 'Length', 'Width', 'Width with mirrors', 'Height', 'Wheelbase', 'Weight Unladen', 'Gross Vehicle Weight', 'Max. Payload', 'Cargo Volume', 'Cargo Volume Max', 'Cargo Volume Frunk', 'Roof Load', 'Tow Hitch Possible', 'Towing Weight Unbraked', 'Towing Weight Braked', 'Vertical Load Max', 'Seats', 'Isofix', 'Turning Circle', 'Platform', 'Car Body', 'Segment', 'Roof Rails', 'EV Dedicated Platform']


In [ ]:
df.to_excel("final.xlsx")